# 데이터 조회
데이터 조회 시 유용한 문법에 대해 알아봅니다.
* [SELECT](#select)
    * [LIMIT](#limit)
    * [연결 연산자 (||)](#연결-연산자)
    * [AS](#as)
* [DISTINCT](#distinct)
* [ORDER BY](#order-by)


> <br/>사용한 데이터 정보  
<br/>
![IMAGE](https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)  
https://www.sqlitetutorial.net/sqlite-sample-database/  
<br/>

## 시작하기 전 실행하기!

In [1]:
import sqlite3
from prettytable import from_db_cursor

# SQLite 데이터베이스에 연결하기 위해 connect 생성
con = sqlite3.connect('./database/chinook.db')
con.row_factory = sqlite3.Row

# cursor 생성
cur = con.cursor()

## SELECT
원하는 데이터를 조회할 때 사용합니다.

※ SQL은 대소문자를 구분하지 않습니다. 명령문을 대문자, 그 외는 소문자로 입력하면 가독성이 좋아집니다.

In [6]:
# 한번에 모든 컬럼 조회
# 데이터의 양이 많은 경우 모든 데이터 조회는 굉장히 오래걸리고 에러가 발생할 수 있어 권장되지 않음
# SELECT * FROM table_name;
cur.execute("""
    SELECT *
    FROM customers
    """)

# 데이터 양이 많으니 상위 5개만 임시로 슬라이싱 해서 보기
# 원하는 수만큼만 보고 싶은 경우는 밑에서 나올 LIMIT이 정석
from_db_cursor(cur)[:5]

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [11]:
# 원하는 컬럼만 지정해서 조회 가능
# SELECT column1, column2 FROM table_name;
cur.execute("""
    SELECT customerid, firstname, city, state
    FROM customers
    """)

from_db_cursor(cur)[:5]

CustomerId,FirstName,City,State
1,Luís,São José dos Campos,SP
2,Leonie,Stuttgart,None
3,François,Montréal,QC
4,Bjørn,Oslo,None
5,František,Prague,None


In [12]:
# 간단한 연산 결과를 얻을 수도 있다
cur.execute("SELECT 1+1")

from_db_cursor(cur)

1+1
2


In [13]:
# 여러개도 가능하다!
cur.execute("SELECT 1+1, 2*5")

from_db_cursor(cur)

1+1,2*5
2,10


In [14]:
# 컬럼끼리 연산하는것도 가능함
cur.execute("""
    SELECT customerid + supportrepid, firstname, city, state
    FROM customers
    """)

from_db_cursor(cur)[:5]

customerid + supportrepid,FirstName,City,State
4,Luís,São José dos Campos,SP
7,Leonie,Stuttgart,None
6,François,Montréal,QC
8,Bjørn,Oslo,None
9,František,Prague,None


### LIMIT
쿼리 결과를 원하는 행 수 만큼만 받도록 합니다.

In [7]:
cur.execute("""
    SELECT *
    FROM customers
    LIMIT 5
    """)

from_db_cursor(cur)

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


OFFSET 키워드로 행을 얼마만큼 건너뛸것인지 지정할 수 있습니다.

In [8]:
# offset을 n으로 지정하면 n행 건너뛰고 n+1행부터 리턴
cur.execute("""
    SELECT *
    FROM customers
    LIMIT 5 OFFSET 10
    """)

from_db_cursor(cur)

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,+55 (11) 3055-8131,alero@uol.com.br,5
12,Roberto,Almeida,Riotur,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,+55 (21) 2271-7000,+55 (21) 2271-7070,roberto.almeida@riotur.gov.br,3
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,+1 (780) 434-5565,mphilips12@shaw.ca,5
15,Jennifer,Peterson,Rogers Canada,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,+1 (604) 688-2255,+1 (604) 688-8756,jenniferp@rogers.ca,3


OFFSET 키워드를 사용하지 않고 지정할 수도 있습니다.

In [9]:
# 앞에 오는 숫자가 offset, 뒤에 오는 숫자가 limit
cur.execute("""
    SELECT *
    FROM customers
    LIMIT 10, 5
    """)

from_db_cursor(cur)

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,+55 (11) 3055-8131,alero@uol.com.br,5
12,Roberto,Almeida,Riotur,"Praça Pio X, 119",Rio de Janeiro,RJ,Brazil,20040-020,+55 (21) 2271-7000,+55 (21) 2271-7070,roberto.almeida@riotur.gov.br,3
13,Fernanda,Ramos,None,Qe 7 Bloco G,Brasília,DF,Brazil,71020-677,+55 (61) 3363-5547,+55 (61) 3363-7855,fernadaramos4@uol.com.br,4
14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,+1 (780) 434-5565,mphilips12@shaw.ca,5
15,Jennifer,Peterson,Rogers Canada,700 W Pender Street,Vancouver,BC,Canada,V6C 1G8,+1 (604) 688-2255,+1 (604) 688-8756,jenniferp@rogers.ca,3


### 연결 연산자 (||)
여러 컬럼의 값 또는 문자열을 합쳐 하나의 컬럼으로 출력합니다.  
문자열은 작은 따옴표(')로 감싸서 입력해야 합니다.

※어떤 DBMS를 사용하는지에 따라 || 연산자로 합치지 못할 수 있습니다.  
  SQlite와 Oracle은 || 로 합칠 수 있지만 MYSQL에서는 CONCAT 함수를 사용해야 합니다.

In [15]:
cur.execute("""
    SELECT firstname||' '||lastname
    FROM customers
    LIMIT 10
    """)

from_db_cursor(cur)

firstname||' '||lastname
Luís Gonçalves
Leonie Köhler
François Tremblay
Bjørn Hansen
František Wichterlová
Helena Holý
Astrid Gruber
Daan Peeters
Kara Nielsen
Eduardo Martins


In [16]:
# 문자열과 합치는 것도 가능
cur.execute("""
    SELECT customerid, 'My name is '||firstname||' '||lastname
    FROM customers
    LIMIT 10
    """)

from_db_cursor(cur)

CustomerId,'My name is '||firstname||' '||lastname
1,My name is Luís Gonçalves
2,My name is Leonie Köhler
3,My name is François Tremblay
4,My name is Bjørn Hansen
5,My name is František Wichterlová
6,My name is Helena Holý
7,My name is Astrid Gruber
8,My name is Daan Peeters
9,My name is Kara Nielsen
10,My name is Eduardo Martins


### AS
컬럼 및 테이블에 임시로 별명을 지어줄 수 있습니다.  

In [17]:
# SELECT column1 AS nickname FROM tablename
cur.execute("""
    SELECT customerid, firstname||' '||lastname AS Name
    FROM customers
    LIMIT 10
    """)

from_db_cursor(cur)

CustomerId,Name
1,Luís Gonçalves
2,Leonie Köhler
3,François Tremblay
4,Bjørn Hansen
5,František Wichterlová
6,Helena Holý
7,Astrid Gruber
8,Daan Peeters
9,Kara Nielsen
10,Eduardo Martins


In [13]:
# 별명에 공백, 특수문자 등을 포함하고 싶은 경우 작은따옴표로 감싸서 문자열 형태로 만들어야 함
cur.execute("""
    SELECT customerid, firstname||' '||lastname AS 'My Name☆'
    FROM customers
    LIMIT 10
    """)

from_db_cursor(cur)

CustomerId,My Name☆
1,Luís Gonçalves
2,Leonie Köhler
3,François Tremblay
4,Bjørn Hansen
5,František Wichterlová
6,Helena Holý
7,Astrid Gruber
8,Daan Peeters
9,Kara Nielsen
10,Eduardo Martins


## DISTINCT
중복된 데이터를 제외하고 유니크한 값만 보여줍니다

In [24]:
# 중복 데이터 예시를 쉽게 보기 위해 데이터 생성
con.close()
con = sqlite3.connect('./database/example.db')
cur = con.cursor()

# 테이블이 이미 있다면 삭제
cur.execute("DROP TABLE IF EXISTS student")

# 테이블 생성
cur.execute("""
    CREATE TABLE student(
        first_name text,
        last_name text,
        age integer
    )
    """)

def data_generator():
    datas = [
                ['Tomas', 'Train', 10],
                ['Bean', 'Green', 1],
                ['Bee', 'Honey', 3],
                ['Bee', 'Honey', 3],
                ['Bee', 'Zig', 5], 
                ['Bob', 'Sponge', 7]
            ]
    for data in datas:
        yield data

# 데이터 입력 
cur.executemany("INSERT INTO student VALUES (?, ?, ?)", data_generator())

# 입력한 데이터 확인
cur.execute("SELECT * FROM student")
from_db_cursor(cur)

first_name,last_name,age
Tomas,Train,10
Bean,Green,1
Bee,Honey,3
Bee,Honey,3
Bee,Zig,5
Bob,Sponge,7


In [25]:
# 원하는 컬럼 앞에 DISTINCT 입력
# SELECT DISTINCT column1 FROM table_name;
cur.execute("""
    SELECT DISTINCT first_name
    FROM student
    """)

from_db_cursor(cur)

first_name
Tomas
Bean
Bee
Bob


In [26]:
# 여러 컬럼을 적을 경우 컬럼의 조합을 기준으로 하기때문에
# 한 컬럼의 값이 중복이어도 다른 컬럼의 값이 다르면 유니크하다고 판단함

# SELECT DISTINCT column1, column2 FROM table_name;
cur.execute("""
    SELECT DISTINCT first_name, last_name
    FROM student
    """)

from_db_cursor(cur)

first_name,last_name
Tomas,Train
Bean,Green
Bee,Honey
Bee,Zig
Bob,Sponge


In [27]:
# 다시 chinook db로 연결
con.close()
con = sqlite3.connect('./database/chinook.db')
cur = con.cursor()

## ORDER BY
특정 컬럼을 기준으로 데이터를 정렬할 수 있습니다.

In [29]:
# SELECT * FROM table_name ORDER BY column1
# LIMIT은 ORDER BY 다음에 작성
cur.execute("""
    SELECT *
    FROM customers
    ORDER BY country
    LIMIT 5
    """)

from_db_cursor(cur)

CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
56,Diego,Gutiérrez,None,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,+54 (0)11 4311 4333,None,diego.gutierrez@yahoo.ar,4
55,Mark,Taylor,None,421 Bourke Street,Sidney,NSW,Australia,2010,+61 (02) 9332 3633,None,mark.taylor@yahoo.au,4
7,Astrid,Gruber,None,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,None,Austria,1010,+43 01 5134505,None,astrid.gruber@apple.at,5
8,Daan,Peeters,None,Grétrystraat 63,Brussels,None,Belgium,1000,+32 02 219 03 03,None,daan_peeters@apple.be,4
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3


In [30]:
# 2개 이상의 컬럼을 정렬 기준으로 넣을 수도 있음
cur.execute("""
    SELECT DISTINCT country, city
    FROM customers
    ORDER BY country, city
    LIMIT 15
    """)

from_db_cursor(cur)

Country,City
Argentina,Buenos Aires
Australia,Sidney
Austria,Vienne
Belgium,Brussels
Brazil,Brasília
Brazil,Rio de Janeiro
Brazil,São José dos Campos
Brazil,São Paulo
Canada,Edmonton
Canada,Halifax


컬럼별로 정렬 순서를 지정할 수 있습니다.
* ASC: 오름차순 (따로 입력하지 않으면 default로 오름차순)
* DESC: 내림차순

In [31]:
# 컬럼 뒤에 정렬 순서 입력하기 (ASC는 생략 가능)
cur.execute("""
    SELECT DISTINCT country, city
    FROM customers
    ORDER BY country ASC, city DESC
    LIMIT 15
    """)

from_db_cursor(cur)

Country,City
Argentina,Buenos Aires
Australia,Sidney
Austria,Vienne
Belgium,Brussels
Brazil,São Paulo
Brazil,São José dos Campos
Brazil,Rio de Janeiro
Brazil,Brasília
Canada,Yellowknife
Canada,Winnipeg
